In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
pip install indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.9 MB/s eta 0:00:00


In [ ]:
!pip install POT==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for POT: filename=POT-0.4.0-cp310-cp310-linux_x86_64.whl size=299608 sha256=59285f711818b93c5ec950137aaa1ddf0fd08917c7ce409242a7882f55c4b0b1
  Stored in directory: /root/.cache/pip/wheels/0e/e4/80/b62808eddaa03369bd2fc227a8cee3f9e8e71b8c22fa49edde
Successfully built POT


In [ ]:
import pandas as pd
import re
import pickle

def extract_sentences(article):
    # Split the article into sentences using regex
    sentences = re.split(r'["।","?","!"]', article)
    # Remove empty strings and strip leading/trailing whitespace
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences



def process_articles(df):
    new_rows = []
    for i, row in df.iterrows():
        article_number = f'{i}'
        sentences = extract_sentences(row['Article'])
        summary_sentence = extract_sentences(row['Summary'])
        tokenized_sentences = sentences + summary_sentence
        model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)
        article_label = []
        for i in range(len(sentences)):
            sent_max = []
            for k in range(len(summary_sentence)):
                wmd_similarity = model.wv.wmdistance(sentences[i], summary_sentence[k])
                sent_max.append(wmd_similarity)
            article_label.append(np.mean(sent_max))
        article_labels = [round(element, 2) for element in article_label]
        print(article_labels)
        threshold = np.mean(article_labels)
        for j, sentence in enumerate(sentences):
            sentence_number = f'S{j+1}'
            if article_label[j] < threshold:
                label = 1
            else:
                label = 0
            new_row = {'id':j,'Article_number': article_number, 'Sentence_number': sentence_number, 'Sentence': sentence, 'Label': label}
            new_rows.append(new_row)
    return pd.DataFrame(new_rows)
# Read the original CSV file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/bengali_train.csv')
df=df.head(10)
# Process articles and create a new DataFrame
new_df = process_articles(df)

# Save the new DataFrame to a CSV file
new_df.to_pickle('/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/processed_Bengali_train.pickle')

[0.21, 0.22, 0.22, 0.2, 0.22, 0.23, 0.2]
[0.03, 0.03, 0.04, 0.04, 0.04, 0.07, 0.05, 0.04, 0.04, 0.06, 0.04, 0.04, 0.04, 0.04, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04, 0.05, 0.04, 0.04, 0.04, 0.04, 0.05, 0.04, 0.05, 0.05, 0.06, 0.04, 0.04, 0.04, 0.05, 0.07, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.05, 0.04, 0.04, 0.04, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04, 0.06, 0.04, 0.06, 0.04, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04, 0.05, 0.05, 0.04, 0.04, 0.04, 0.05, 0.06]
[0.5, 0.47, 0.65, 0.65, 0.59, 0.63, 0.58, 0.63, 0.68, 0.64, 0.6, 0.66]


[0.22, 0.2, 0.22, 0.21, 0.19, 0.19, 0.19, 0.19, 0.16, 0.17, 0.18, 0.21, 0.23, 0.25, 0.22, 0.18, 0.22, 0.2, 0.19, 0.26, 0.24, 0.23, 0.22, 0.19, 0.2, 0.19, 0.19, 0.21, 0.25, 0.19, 0.2, 0.25, 0.18, 0.41]
[0.08, 0.08, 0.08, 0.08, 0.07, 0.06, 0.08, 0.1, 0.07, 0.08, 0.09, 0.08, 0.09, 0.08, 0.11, 0.09, 0.1, 0.08, 0.07, 0.07, 0.11, 0.09, 0.1, 0.09, 0.07, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08]
[0.3, 0.35, 0.31, 0.24, 0.39, 0.37, 0.3, 0.3, 0.37, 0.36, 0.28, 0.37, 0.34, 0.31, 0.36, 0.3, 0.35, 0.33, 0.36, 0.39, 0.37, 0.37, 0.39, 0.45, 0.37, 0.46, 0.37, 0.41, 0.34, 0.34, 0.3, 0.4, 0.34, 0.32, 0.34, 0.32, 0.37, 0.35, 0.38, 0.34, 0.49, 0.42, 0.38, 0.53]
[0.37, 0.36, 0.43, 0.35, 0.44, 0.74, 0.43, 0.42, 0.43, 0.41, 0.42, 0.43, 0.54, 0.45, 0.44, 0.39, 0.41, 0.41, 0.55, 0.4, 0.57, 0.38, 0.59]


[0.1, 0.09, 0.13, 0.11, 0.11, 0.11, 0.14, 0.24, 0.13, 0.12, 0.11, 0.11, 0.15, 0.12, 0.17, 0.14, 0.13, 0.11, 0.18, 0.11, 0.12, 0.11, 0.23, 0.11, 0.11, 0.12, 0.12, 0.13, 0.12, 0.12, 0.12, 0.11, 0.11, 0.19]
[0.11, 0.12, 0.12, 0.14, 0.12, 0.12, 0.11, 0.11, 0.12, 0.11, 0.08, 0.1, 0.1, 0.1, 0.11, 0.1, 0.11, 0.11, 0.12, 0.1, 0.12, 0.11, 0.11, 0.11, 0.09, 0.12, 0.13, 0.11, 0.12, 0.13, 0.12, 0.11, 0.1, 0.11, 0.11]
[0.23, 0.25, 0.21, 0.2, 0.23, 0.24, 0.32, 0.37, 0.23, 0.26, 0.28, 0.23, 0.24, 0.24, 0.26, 0.23, 0.25, 0.23, 0.23, 0.24, 0.28, 0.25, 0.24, 0.29, 0.23, 0.24, 0.23, 0.24, 0.36]


In [ ]:
pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/processed_Bengali_train.pickle')

,id,Article_number,Sentence_number,Sentence,Label
0,0,0,S1,শ্রীহরিকোটা: ভারতের মহাকাশ বিজ্ঞানে আজ এক দার...,0
1,1,0,S2,রোভার এবং একটি প্রোপালশন মডিউল রয়ছে৷ যার ওজন ...,1
2,2,0,S3,আবহাওয়ার বড় তোলপাড়ের ইঙ্গিত কি%%%%%%%%রোভার...,1
3,3,0,S4,যা চন্দ্রযান ২-র প্রজ্ঞান রোভারের সমান ওজনের৷ ...,0
4,4,0,S5,রোভারে হবে ৫০ ওয়াট৷%%%%চন্দ্রায়ন -৩,1
...,...,...,...,...,...
319,24,9,S25,রামগঙ্গা রেঞ্জের অধীনে ১১০০ বর্গ কিলোমিটার এলা...,0
320,25,9,S26,অপরদিকে ন্যাশানাল পার্ক ইস্ট,0
321,26,9,S27,ন্যাশানাল পার্ক ওয়েস্ট,0
322,27,9,S28,সজনেখালি ও বসিরহাট রেঞ্জের ২৫০০ বর্গকিলোমিটার ...,0


In [ ]:
dataset = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/processed_Bengali_train.pickle')

In [ ]:
print(len(dataset))

324


In [ ]:
from indicnlp.tokenize import indic_tokenize
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# Load the dataset
#dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Minor Project/Bengali/NewBengali_dataset.csv')

# Tokenize sentences
tokenized_sentences = []
for sentence in dataset['Sentence']:
    try:
        # Check if the sentence is a string (to handle NaN values)
        if isinstance(sentence, str):
            tokens = indic_tokenize.trivial_tokenize(sentence)
            tokenized_sentences.append(tokens)
    except TypeError:
        # Handle non-string values
        pass

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained Word2Vec model
model.save('/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/bengali_word2vec.model')

# Calculate sentence embeddings
sentence_embeddings = []
for tokens in tokenized_sentences:
    word_embeddings = []
    for token in tokens:
        try:
            word_embedding = model.wv[token]
            word_embeddings.append(word_embedding)
        except KeyError:
            # Handle out-of-vocabulary words
            pass
    if word_embeddings:
        sentence_embedding = np.mean(word_embeddings, axis=0)
    else:
        # Append a zero vector of the same dimension as the embeddings
        sentence_embedding = np.zeros(model.vector_size)
    sentence_embeddings.append(sentence_embedding)





In [ ]:
# Calculate sentence embeddings
sentence_embeddings = []
for tokens in tokenized_sentences:
    word_embeddings = []
    for token in tokens:
        try:
            word_embedding = model.wv[token]
            word_embeddings.append(word_embedding)
        except KeyError:
            # Handle out-of-vocabulary words
            pass
    if word_embeddings:
        sentence_embedding = np.mean(word_embeddings, axis=0)
        sentence_embeddings.append(sentence_embedding)
    else:
        print("No word embeddings found for tokens:", tokens)

print("Number of sentences with embeddings:", len(sentence_embeddings))
print("Number of sentences in dataset:", len(dataset))


Number of sentences with embeddings: 324
Number of sentences in dataset: 324


In [ ]:
print(len(sentence_embeddings))
print(len(dataset))

324
324


In [ ]:
sentence_embedding


array([-0.00122102, -0.00095307,  0.00328747,  0.00648924, -0.00612718,
       -0.00895291,  0.00255453,  0.00762113, -0.00357348, -0.00390212,
        0.00492153, -0.00188323, -0.00241797,  0.00227476, -0.00336998,
       -0.00385777,  0.00324993,  0.00171528, -0.00515771, -0.01027994,
        0.0052326 ,  0.00621365,  0.00341585, -0.00036513,  0.00353639,
        0.00024874, -0.00303407,  0.00208329, -0.00352375, -0.00287684,
       -0.00626297, -0.00314337,  0.00681585, -0.00304296, -0.00322668,
       -0.00132059,  0.00427677, -0.00418246, -0.00070671, -0.0023598 ,
       -0.00488434,  0.0022037 , -0.00362404, -0.00321136,  0.00269888,
       -0.00195258, -0.0062331 ,  0.00331573,  0.00232171,  0.00719315,
       -0.00395537,  0.00079255, -0.00436025,  0.00123624,  0.00465507,
        0.00073167,  0.00275473, -0.00302968, -0.00452238,  0.00432004,
        0.00226728,  0.00285813, -0.00179157, -0.00140307, -0.00428538,
        0.00172259, -0.00136631,  0.00427679, -0.00326367,  0.00

In [ ]:

# Add the sentence_embeddings as a new column to the dataset
dataset['sentence_embedding'] = sentence_embeddings

# Save the updated dataset
output_path = '/content/drive/MyDrive/Colab Notebooks/Minor Project/Final_create_dataset_for_bengali/Final_dataset.csv'
dataset.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
from gensim.models import Word2Vec
import csv
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
dataset

,id,Article_number,Sentence_number,Sentence,Label,sentence_embedding
0,0,0,S1,শ্রীহরিকোটা: ভারতের মহাকাশ বিজ্ঞানে আজ এক দার...,0,"[-0.0006313565, 0.0003912982, 0.0016184599, 0...."
1,1,0,S2,রোভার এবং একটি প্রোপালশন মডিউল রয়ছে৷ যার ওজন ...,1,"[-0.0004835508, -0.00029880443, -0.00013506752..."
2,2,0,S3,আবহাওয়ার বড় তোলপাড়ের ইঙ্গিত কি%%%%%%%%রোভার...,1,"[0.0011581162, 0.00066569226, 0.0024635314, 0...."
3,3,0,S4,যা চন্দ্রযান ২-র প্রজ্ঞান রোভারের সমান ওজনের৷ ...,0,"[-0.0016089833, -4.8101065e-05, 0.0021922325, ..."
4,4,0,S5,রোভারে হবে ৫০ ওয়াট৷%%%%চন্দ্রায়ন -৩,1,"[-0.00040632693, 0.0030558745, 0.0032430096, 0..."
...,...,...,...,...,...,...
319,24,9,S25,রামগঙ্গা রেঞ্জের অধীনে ১১০০ বর্গ কিলোমিটার এলা...,0,"[-0.0009635572, -0.00052576105, 0.0013313509, ..."
320,25,9,S26,অপরদিকে ন্যাশানাল পার্ক ইস্ট,0,"[-0.0013055751, 0.0016489845, -0.00029905594, ..."
321,26,9,S27,ন্যাশানাল পার্ক ওয়েস্ট,0,"[0.0020518587, 0.004703524, 0.0008928071, -0.0..."
322,27,9,S28,সজনেখালি ও বসিরহাট রেঞ্জের ২৫০০ বর্গকিলোমিটার ...,0,"[0.0014341739, 0.002470043, -0.00019818242, -0..."
